In [1]:
import os
import time
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt
from time import perf_counter
from IPython.display import clear_output
from pathlib import Path

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

from tqdm.notebook import tqdm
import datetime


%matplotlib qt


%load_ext autoreload
%autoreload 2


C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
####      INIT BOARD        #######
BoardShim.enable_dev_board_logger()
params = BrainFlowInputParams()
board = BoardShim(BoardIds.MINDROVE_WIFI_BOARD, params)

try:
    board.stop_stream()
    board.release_session()
except:
    ...
    
board.prepare_session()
sample_rate = board.get_sampling_rate(16)
print("Device ready (sampling rate: {}hz)".format(sample_rate))

Device ready (sampling rate: 500hz)


In [3]:
def visualize(data):
   
    fig = plt.gcf()
    fig.canvas.manager.window.raise_()
    mngr = plt.get_current_fig_manager()
    
 
 
    plt.cla()
    plt.ylim(-1, 1)
    for i, c in enumerate(data):
        plt.plot(c)
    

    mngr.window.setGeometry(1150,100,800, 800)
    fig.canvas.flush_events()
    plt.draw()
    # plt.pause(0.001)
    #plt.cla()
    plt.show()
    
def get_avg(data):
    
    
    mins = []
    for i, channel in enumerate(data):
        mins.append(np.min(channel))
    data = np.delete(data, np.argmin(mins), 0)

    maxs = []
    for i, channel in enumerate(data):
        maxs.append(np.max(channel))
    data = np.delete(data, np.argmax(maxs), 0)
    
    return data


def DCFilter(data):
    #return data - np.mean(data, axis=0)
    new_data = []
    for d in data:
        new_data.append(d - np.mean(d))
    return np.array(new_data)

In [4]:
clip_value = 2000

board.start_stream(450000) 

time.sleep(10)

for i in range(1000):
    clear_output()
    time.sleep(0.1)
    
    data = board.get_current_board_data(sample_rate*10) # 10 seconds
    data = DCFilter(data)

    data = data[:6,-500:]
    data = get_avg(data)
    data = np.clip(data, -clip_value, clip_value)
    data /= clip_value
    
    visualize(data)